In [1]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
import cv2
import numpy as np
from sklearn import svm
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

import multiprocessing as mp
from tqdm import tqdm

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score

In [2]:
# Binary feather files
train_f = pd.read_feather("../data/binary/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f = pd.read_feather("../data/binary/val_all_features.f", columns=None, use_threads=True, storage_options=None)
train_shape = train_f.shape
val_shape = val_f.shape
print(f'train shape: {train_shape}, val shape: {val_shape}')
# Multi-class feather files.
train_f_mul = pd.read_feather("../data/three_class/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f_mul = pd.read_feather("../data/three_class/val_all_feat.f", columns=None, use_threads=True, storage_options=None)
train_shape_m = train_f_mul.shape
val_shape_m = val_f_mul.shape
print(f'train shape multi: {train_shape_m}, val shape multi: {val_shape_m}')

train shape: (15195, 144), val shape: (3796, 144)
train shape multi: (5082, 144), val shape multi: (1270, 144)


In [12]:
## Pre-processing
# Binary case
train_f.replace({'nevus': 1, 'others': 0}, inplace=True)
val_f.replace({'nevus': 1, 'others': 0}, inplace=True)

df_train = train_f.iloc[:train_shape[0], :].sample(frac=1, random_state=42)
df_val = val_f.iloc[:val_shape[0], :].sample(frac=1, random_state=42)

X_train, y_train = df_train.iloc[:, :(train_shape[1]-1)].to_numpy(dtype=np.float32), df_train.iloc[:, (train_shape[1]-1)].to_numpy()
X_test, y_test = df_val.iloc[:, :(val_shape[1]-1)].to_numpy(dtype= np.float32), df_val.iloc[:, (val_shape[1]-1)].to_numpy()

print(f'X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}')

X_train: (15195, 143), y_train: (15195,), X_test: (3796, 143), y_test: (3796,)


In [13]:
# SVM
# param_grid = {'classifier__C': [0.1, 1],
#               'classifier__gamma': [1, 0.1],
#               'classifier__kernel': ['rbf', 'linear', 'poly']}

param_grid = {'classifier__C': [1],
              'classifier__gamma': [0.1],
              'classifier__kernel': ['rbf']}
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
classifier = svm.SVC()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 3)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

Fitting 10 folds for each of 75 candidates, totalling 750 fits
[CV 1/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.509 total time= 1.1min
[CV 2/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.509 total time= 1.1min
[CV 3/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.509 total time= 1.1min
[CV 4/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.509 total time= 1.1min
[CV 5/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.509 total time= 1.1min
[CV 6/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.508 total time= 1.1min
[CV 7/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.508 total time= 1.1min
[CV 8/10] END classifier__C=0.1, classifier__gamma=1, classifier__kernel=rbf;, score=0.508 total time= 1.1min
[CV 9/10] END classifier__C=0.1, classifier__gamma=1, cla

KeyboardInterrupt: 

In [14]:
param_grid = {'classifier__C': [1],
              'classifier__gamma': [0.1],
              'classifier__kernel': ['rbf']}
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
classifier = svm.SVC()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 3)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV 1/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.825 total time= 1.5min
[CV 2/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.830 total time= 1.4min
[CV 3/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.839 total time= 1.4min
[CV 4/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.839 total time= 1.4min
[CV 5/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.824 total time= 1.5min
[CV 6/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.849 total time= 1.5min
[CV 7/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.825 total time= 1.4min
[CV 8/10] END classifier__C=1, classifier__gamma=0.1, classifier__kernel=rbf;, score=0.831 total time= 1.4min
[CV 9/10] END classifier__C=1, classifier__gamma=0.1, class